# Определение текущего уровня потребительской лояльности (NPS) в телекоммуникационной компании

**Описание проекта**

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 


Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».


Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 

Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса. 


**Цели и задачи проекта**

Цель проекта - визуализировать результаты опроса телекоммуникационной компании в виде дашборда.

Задачи проекта:

1. Полключиться к базе данных
2. Выгрузить данные с помощью SQL-запроса
3. Провести обработку данных и исследовательский анализ в Tableau
4. Составить подходящие визуализации 
5. Завершить анализ созданием дашборда

In [14]:
# импортируем библиотеки
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [15]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [16]:
query = """
SELECT 
    u.user_id,
    u.lt_day,
    CASE 
        WHEN u.lt_day <= 365 THEN 1
        ELSE 0
    END AS is_new,
    u.age,
    CASE 
        WHEN gender_segment = 1 THEN 'Female'
        WHEN gender_segment = 0 THEN 'Male'
        ELSE 'Unknown'
    END AS gender_segment,
    u.os_name,
    u.cpe_type_name,
    l.country,
    l.city,
    a.title AS age_segment,
    t.title AS traffic_segment,
    lt.title AS lifetime_segment,
    u.nps_score,
    CASE 
        WHEN u.nps_score >= 9 THEN 'promoter'
        WHEN u.nps_score >= 7 THEN 'passive'
        ELSE 'detractor'
    END AS nps_group
FROM user u
JOIN location l ON u.location_id = l.location_id
JOIN age_segment a ON u.age_gr_id = a.age_gr_id
JOIN traffic_segment t ON u.tr_gr_id = t.tr_gr_id
JOIN lifetime_segment lt ON u.lt_gr_id = lt.lt_gr_id;

"""

In [17]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,Female,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoter
1,A001WF,2344,0,53.0,Male,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoter
2,A003Q7,467,0,57.0,Male,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoter


In [18]:
# изучим датафрейм
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  int64  
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(3), object(10)
memory usage: 53.7+ MB


In [19]:
# изучим датафрейм
df.describe()

,lt_day,is_new,age,nps_score
count,502493.000000,502493.000000,501939.000000,502493.000000
mean,1868.841439,0.171346,39.621946,7.508562
std,1683.701762,0.376811,11.188249,3.020378
min,-21.000000,0.000000,10.000000,1.000000
25%,533.000000,0.000000,31.000000,5.000000
50%,1239.000000,0.000000,38.000000,9.000000
75%,3064.000000,0.000000,47.000000,10.000000
max,9162.000000,1.000000,89.000000,10.000000


В данных 14 колонок, 502493 строк. 

Типы данных: float64(1), int64(3), object(10)

Память: 53.7+ MB

In [20]:
file_path = 'filename.csv'

try:
    df.to_csv(file_path, index=False)
    print(f"File saved successfully to {file_path}")
except FileNotFoundError:
    print("File path is incorrect or directories do not exist.")
except PermissionError:
    print("Permission denied. Check your write permissions.")
except Exception as e:
    print(f"An error occurred: {e}")


File saved successfully to filename.csv
